In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
df = pd.read_csv('all_auction_add_close.csv', encoding = 'utf8')
# df.head()

## 整理資料
* to_datetime
* 刪掉重複資料
* 刪除競拍失敗/尚未完成的資料
* label stock & cb

In [41]:
df['開標日期'] = pd.to_datetime(df['開標日期'])
# df['開標日期']

# 刪掉重複的一筆(泰昇-ky, 聯廣)
index_name = df[(df['得標總金額(元)'] == '0') & (df['證券名稱'] =='泰昇-KY')].index
df.drop(index_name, inplace = True)
df.reset_index()
# df[df['證券名稱'] =='泰昇-KY']
index_name = df[(df['得標總金額(元)'] == '0') & (df['證券名稱'] =='聯廣')].index
df.drop(index_name, inplace = True)
df.reset_index()
# df[df['證券名稱'] =='聯廣']

index_name = df[(df['得標總金額(元)'] == '0')].index
df.drop(index_name, inplace = True)

def label_stock(row):
    if row['發行性質'] in ['有擔保轉換公司債','無擔保轉換公司債']:
        return "cb"
    else:
        return "stock" 
df['類別'] = df.apply(lambda row : label_stock(row), axis = 1)
print('stock筆數 =', sum(df['類別'] == 'stock'))
print('cb筆數 =', sum(df['類別'] == 'cb'))

stock筆數 = 173
cb筆數 = 26


## 針對stock再進一步處理
* 刪除第一上市初上市, 第一上櫃初上櫃資料(因為沒有投標最後一天成交價)
* 刪除有遺漏值的row


In [42]:
df_stock = df[df['類別'] == 'stock']
df_stock.drop(['可轉債個股投標最後一天均價'], axis = 1, inplace = True)
print(df_stock.shape) # 173 * 25

# 刪除第一上市初上市, 第一上櫃初上櫃, 共9筆
index = df_stock[(df_stock['發行性質'] == '第一上市初上市') | (df_stock['發行性質'] == '第一上櫃初上櫃')].index
df_stock.drop(index, inplace = True)
# print(df_stock.shape) # 148 * 25
# df_stock.info() # nan: (投標最後一天平均成交價 * 1, 撥券日開盤價 * 3)

# 直接把有遺漏值的row都刪掉
df_stock.dropna(inplace = True)
# print(df_stock.shape) # 144 * 25

(173, 25)


## 準備訓練資料
* (刪除有outlier的row)
* 轉換資料型別

In [ ]:
train_col = ['開標日期','證券名稱', '證券代號', '發行市場', '發行性質', '競拍數量(張)','類別', '最低投標價格(元)', '最低每標單投標數量(張)','每一投標單投標處理費(元)', '得標手續費率(%)', '投標最後一天平均成交價', '最低得標價格(元)']

## 切割訓練資料(使用前121筆, 70%)

121.1